In [2]:
import os
import re
import pandas as pd
from pathlib import Path
from PIL import Image
import shutil

# زيادة حد الأمان للصور عالية الدقة
Image.MAX_IMAGE_PIXELS = 1000000000  # 1 بليون بكسل

# المسار الرئيسي
main_path = r"C:\Users\eyadn\Downloads\BPA\gun shot"

# إنشاء مجلد في Desktop للصور المعالجة
desktop_path = os.path.expanduser("~\\Desktop")
output_images_dir = os.path.join(desktop_path, "gun_images_224x224")
if os.path.exists(output_images_dir):
    shutil.rmtree(output_images_dir)
os.makedirs(output_images_dir)

# قائمة لتخزين النتائج
data = []

# البحث عن الـ 5 ملفات الرئيسية
subdirs = [d for d in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, d))]

for subdir in subdirs[:5]:  # أول 5 ملفات
    subdir_path = os.path.join(main_path, subdir)
    
    # البحث عن جميع الملفات داخل المجلد الفرعي
    for root, dirs, files in os.walk(subdir_path):
        # البحث عن ملفات JPG و Text معاً
        jpg_files = [f for f in files if f.lower().endswith('.jpg')]
        txt_files = [f for f in files if f.lower().endswith('.txt')]
        
        # معالجة كل صورة JPG
        for jpg_file in jpg_files:
            jpg_name = jpg_file
            jpg_path = os.path.join(root, jpg_file)
            gun_class = "غير محدد"  # القيمة الافتراضية
            
            # البحث عن ملف نص مطابق
            for txt_file in txt_files:
                txt_path = os.path.join(root, txt_file)
                
                try:
                    # محاولة قراءة الملف بـ encodings مختلفة
                    encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1', 'utf-16']
                    content = None
                    
                    for encoding in encodings:
                        try:
                            with open(txt_path, 'r', encoding=encoding) as f:
                                content = f.read()
                            break
                        except:
                            continue
                    
                    if content:
                        # البحث عن "Gun:" وحفظ القيمة
                        match = re.search(r'Gun:\s*(.+?)(?:\n|$)', content)
                        if match:
                            gun_class = match.group(1).strip()
                            break  # استخراج القيمة من أول ملف نص
                
                except Exception as e:
                    print(f"خطأ في قراءة الملف {txt_file}: {e}")
            
            # معالجة الصورة وتحويلها إلى 224x224
            try:
                img = Image.open(jpg_path)
                # تحويل إلى RGB في حالة صورة RGBA
                if img.mode == 'RGBA':
                    img = img.convert('RGB')
                # تعديل حجم الصورة
                img_resized = img.resize((224, 224), Image.Resampling.LANCZOS)
                
                # حفظ الصورة المعالجة
                output_image_path = os.path.join(output_images_dir, jpg_name)
                img_resized.save(output_image_path, quality=95)
                
            except Exception as e:
                print(f"خطأ في معالجة الصورة {jpg_name}: {e}")
            
            # إضافة البيانات للقائمة
            data.append({
                'اسم الصورة': jpg_name,
                'الكلاس (Gun)': gun_class
            })

# إنشاء DataFrame
df = pd.DataFrame(data)

# حفظ في ملف Excel على Desktop
excel_output_path = os.path.join(desktop_path, 'gun_classes.xlsx')
df.to_excel(excel_output_path, index=False, sheet_name='البيانات')

print(f"✓ تم حفظ البيانات بنجاح في: {excel_output_path}")
print(f"✓ تم حفظ الصور (224x224) في: {output_images_dir}")
print(f"✓ عدد الصور المعالجة: {len(data)}")
print("\nعينة من البيانات:")
print(df.head(10))

c:\Users\eyadn\anaconda3\Lib\site-packages\PIL\Image.py:3402: DecompressionBombWarning: Image size (1200000000 pixels) exceeds limit of 1000000000 pixels, could be decompression bomb DOS attack.
  warnings.warn(


✓ تم حفظ البيانات بنجاح في: C:\Users\eyadn\Desktop\gun_classes.xlsx
✓ تم حفظ الصور (224x224) في: C:\Users\eyadn\Desktop\gun_images_224x224
✓ عدد الصور المعالجة: 70

عينة من البيانات:
  اسم الصورة                  الكلاس (Gun)
0  Hr24#.jpg  Smith and Wesson handgun 9mm
1   Hr35.jpg  Smith and Wesson handgun 9mm
2   Hr36.jpg  Smith and Wesson handgun 9mm
3   Rp41.jpg  Rock River Arms rifle .223
4   Rp42.jpg  Rock River Arms rifle .223
5   Rp43.jpg  Rock River Arms rifle .223
6   Rp44.jpg  Rock River Arms rifle .223
7   Rp45.jpg  Rock River Arms rifle .223
8   Rp46.jpg  Rock River Arms rifle .223
9   Rp47.jpg  Rock River Arms rifle .223
